# Preprocessing SNP array data - step2

## PLINK pre-imputation QC

Generate binary ped file & remove file with low gentrain score (cutoff = 0.7)

In [1]:
### case
!conda run -n snp_array bash -c 'plink --file data/Case_GSE69664_GPL19864_FW_Raw \
    --exclude data/Gentrain_GSE69664_GPL19864_Excluder.txt \
    --nonfounders --allow-no-sex \
    --allow-extra-chr --chr 1-22 --snps-only \
    --make-bed --out data/Case_GSE69664_GPL19864_Step1'

!conda run -n snp_array bash -c 'plink --file data/Case_GSE69664_GPL20166_FW_Raw \
    --exclude data/Gentrain_GSE69664_GPL20166_Excluder.txt \
    --nonfounders --allow-no-sex \
    --allow-extra-chr --chr 1-22 --snps-only \
    --make-bed --out data/Case_GSE69664_GPL20166_Step1'

### control
!conda run -n snp_array bash -c 'plink --file data/Control_GSE141901_FW_Raw \
    --exclude data/Gentrain_GSE141901_Excluder.txt \
    --nonfounders --allow-no-sex \
    --allow-extra-chr --chr 1-22 --snps-only \
    --make-bed --out data/Control_GSE141901_Step1'

!conda run -n snp_array bash -c 'plink --file data/Control_GSE248483_FW_Raw \
    --exclude data/Gentrain_GSE248483_Excluder.txt \
    --nonfounders --allow-no-sex \
    --allow-extra-chr --chr 1-22 --snps-only \
    --make-bed --out data/Control_GSE248483_Step1'

!conda run -n snp_array bash -c 'plink --file data/Control_GSE74100_FW_Raw \
    --exclude data/Gentrain_GSE74100_Excluder.txt \
    --nonfounders --allow-no-sex \
    --allow-extra-chr --chr 1-22 --snps-only \
    --make-bed --out data/Control_GSE74100_Step1'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/Case_GSE69664_GPL19864_Step1.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --chr 1-22
  --exclude data/Gentrain_GSE69664_GPL19864_Excluder.txt
  --file data/Case_GSE69664_GPL19864_FW_Raw
  --make-bed
  --nonfounders
  --out data/Case_GSE69664_GPL19864_Step1
  --snps-only

362751 MB RAM detected; reserving 181375 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (868123 variants, 493 people).
--file: data/Case_GSE69664_GPL19864_Step1-temporary.bed +
data/Case_GSE69664_GPL19864_Step1-temporary.bim +
data/Case_GSE69664_GPL19864_Step1-temporary.fam written.
868123 variants loaded from .bim file.
493 people (0 males, 0 females, 493 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/Case_GSE69664_GPL19864_Step1.nosex .
--exclude: 851420 v


Add phenotype (control=1, case=2)

In [2]:
%%bash
mv data/Case_GSE69664_GPL19864_Step1.fam data/Case_GSE69664_GPL19864_Step1_NoPheno.fam
mv data/Case_GSE69664_GPL20166_Step1.fam data/Case_GSE69664_GPL20166_Step1_NoPheno.fam
mv data/Control_GSE141901_Step1.fam data/Control_GSE141901_Step1_NoPheno.fam
mv data/Control_GSE248483_Step1.fam data/Control_GSE248483_Step1_NoPheno.fam
mv data/Control_GSE74100_Step1.fam data/Control_GSE74100_Step1_NoPheno.fam

awk '{print "MHS", $2, $3, $4, $5, "2"}' data/Case_GSE69664_GPL19864_Step1_NoPheno.fam > data/Case_GSE69664_GPL19864_Step2.fam
awk '{print "MHS", $2, $3, $4, $5, "2"}' data/Case_GSE69664_GPL20166_Step1_NoPheno.fam > data/Case_GSE69664_GPL20166_Step2.fam
awk '{print "MHS", $2, $3, $4, $5, "1"}' data/Control_GSE141901_Step1_NoPheno.fam > data/Control_GSE141901_Step2.fam
awk '{print "MHS", $2, $3, $4, $5, "1"}' data/Control_GSE248483_Step1_NoPheno.fam > data/Control_GSE248483_Step2.fam
awk '{print "MHS", $2, $3, $4, $5, "1"}' data/Control_GSE74100_Step1_NoPheno.fam > data/Control_GSE74100_Step2.fam

rm data/Case_GSE69664_GPL19864_Step1_NoPheno.fam
rm data/Case_GSE69664_GPL20166_Step1_NoPheno.fam
rm data/Control_GSE141901_Step1_NoPheno.fam
rm data/Control_GSE248483_Step1_NoPheno.fam
rm data/Control_GSE74100_Step1_NoPheno.fam

In [4]:
%%bash
# Update bed and bim files to Step2
for dataset in Case_GSE69664_GPL19864 Case_GSE69664_GPL20166 Control_GSE141901 Control_GSE248483 Control_GSE74100; do
    cp data/${dataset}_Step1.bed data/${dataset}_Step2.bed
    cp data/${dataset}_Step1.bim data/${dataset}_Step2.bim
done

PLINK quality control

In [5]:
# --geno 0.05:  SNP call rate >95%
# --mind 0.01:  sample call rate >99% (we use 0.2 as some dataset have lower genotype rate)
# --maf 0.01:   kept SNPs with minor allele frequencies > 0.01 
# --hwe 0.0001: threshold for Hardy–Weinberg equilibrium of 1e-4 in control cohort 

# update snp rsid for human omnizhonghua and asian screening array platform

!conda run -n snp_array bash -c 'plink --bfile data/Case_GSE69664_GPL19864_Step2 \
    --geno 0.05 --mind 0.2 --maf 0.01 --hwe 0.0001 \
    --nonfounders --allow-no-sex \
    --update-name data/SNPID_HumanOmniZhonghua_OldnNew.txt \
    --make-bed --out data/Case_GSE69664_GPL19864_Step3'

!conda run -n snp_array bash -c 'plink --bfile data/Case_GSE69664_GPL20166_Step2 \
    --geno 0.05 --mind 0.2 --maf 0.01 --hwe 0.0001 \
    --nonfounders --allow-no-sex \
    --update-name data/SNPID_HumanOmniZhonghua_OldnNew.txt \
    --make-bed --out data/Case_GSE69664_GPL20166_Step3'

!conda run -n snp_array bash -c 'plink --bfile data/Control_GSE141901_Step2 \
    --geno 0.05 --mind 0.2 --maf 0.01 --hwe 0.0001 \
    --nonfounders --allow-no-sex \
    --update-name data/SNPID_AsianScreeningArray_OldnNew.txt \
    --make-bed --out data/Control_GSE141901_Step3'

!conda run -n snp_array bash -c 'plink --bfile data/Control_GSE248483_Step2 \
    --geno 0.05 --mind 0.2 --maf 0.01 --hwe 0.0001 \
    --nonfounders --allow-no-sex \
    --update-name data/SNPID_AsianScreeningArray_OldnNew.txt \
    --make-bed --out data/Control_GSE248483_Step3'

!conda run -n snp_array bash -c 'plink --bfile data/Control_GSE74100_Step2 \
    --geno 0.05 --mind 0.2 --maf 0.01 --hwe 0.0001 \
    --nonfounders --allow-no-sex \
    --update-name data/SNPID_HumanOmniZhonghua_OldnNew.txt \
    --make-bed --out data/Control_GSE74100_Step3'


PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/Case_GSE69664_GPL19864_Step3.log.
Options in effect:
  --allow-no-sex
  --bfile data/Case_GSE69664_GPL19864_Step2
  --geno 0.05
  --hwe 0.0001
  --maf 0.01
  --make-bed
  --mind 0.2
  --nonfounders
  --out data/Case_GSE69664_GPL19864_Step3
  --update-name data/SNPID_HumanOmniZhonghua_OldnNew.txt

362751 MB RAM detected; reserving 181375 MB for main workspace.
851420 variants loaded from .bim file.
493 people (0 males, 0 females, 493 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/Case_GSE69664_GPL19864_Step3.nosex .
493 phenotype values loaded from .fam.
--update-name: 851420 values updated, 43121 variant IDs not present.
0 people removed due to missing genotype data (--mind).
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 493 founders and 0 nonfounders present.


## PLINK to VCF

In [6]:
# Reference:
# https://genome.sph.umich.edu/wiki/Minimac3_Cookbook_:_Converting_Files_to_VCF
# https://www.cog-genomics.org/plink/1.9/data#recode

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    plink --bfile data/Case_GSE69664_GPL19864_Step3 --nonfounders --allow-no-sex --chr $chr --snps-only just-acgt --recode vcf --out data/Case_GSE69664_GPL19864_Step3_${chr}; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    plink --bfile data/Case_GSE69664_GPL20166_Step3 --nonfounders --allow-no-sex --chr $chr --snps-only just-acgt --recode vcf --out data/Case_GSE69664_GPL20166_Step3_${chr}; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    plink --bfile data/Control_GSE141901_Step3 --nonfounders --allow-no-sex --chr $chr --snps-only just-acgt --recode vcf --out data/Control_GSE141901_Step3_${chr}; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    plink --bfile data/Control_GSE248483_Step3 --nonfounders --allow-no-sex --chr $chr --snps-only just-acgt --recode vcf --out data/Control_GSE248483_Step3_${chr}; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    plink --bfile data/Control_GSE74100_Step3 --nonfounders --allow-no-sex --chr $chr --snps-only just-acgt --recode vcf --out data/Control_GSE74100_Step3_${chr}; done'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/Case_GSE69664_GPL19864_Step3_1.log.
Options in effect:
  --allow-no-sex
  --bfile data/Case_GSE69664_GPL19864_Step3
  --chr 1
  --nonfounders
  --out data/Case_GSE69664_GPL19864_Step3_1
  --recode vcf
  --snps-only just-acgt

362751 MB RAM detected; reserving 181375 MB for main workspace.
62536 out of 789893 variants loaded from .bim file.
493 people (0 males, 0 females, 493 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/Case_GSE69664_GPL19864_Step3_1.nosex .
493 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 493 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is 0.999307.
62536 variants and 493 people pass filters and QC.
Among remaining phenotypes, 493 are cases and 0 are cont

In [7]:
# bgzip all vcf files
!conda run -n snp_array bash -c 'parallel bgzip {} ::: data/*.vcf'

## beagle5.4 (phasing and imputation)

In [8]:
!bash -c 'for chr in {1..22}; do \
    java -Xmx200g -jar utils/ProgramFiles_Beagle5.4/beagle.22Jul22.46e.jar \
    gt=data/Case_GSE69664_GPL19864_Step3_${chr}.vcf.gz out=data/Case_GSE69664_GPL19864_Step3_imputed_${chr} \
    ref=utils/ProgramFiles_Beagle5.4/chr${chr}.1kg.phase3.v5a.b37.bref3 \
    map=utils/ProgramFiles_Beagle5.4/plink.chr${chr}.GRCh37.map \
    chrom="$chr" nthreads=20; done'

!bash -c 'for chr in {1..22}; do \
    java -Xmx200g -jar utils/ProgramFiles_Beagle5.4/beagle.22Jul22.46e.jar \
    gt=data/Case_GSE69664_GPL20166_Step3_${chr}.vcf.gz out=data/Case_GSE69664_GPL20166_Step3_imputed_${chr} \
    ref=utils/ProgramFiles_Beagle5.4/chr${chr}.1kg.phase3.v5a.b37.bref3 \
    map=utils/ProgramFiles_Beagle5.4/plink.chr${chr}.GRCh37.map \
    chrom="$chr" nthreads=20; done'

!bash -c 'for chr in {1..22}; do \
    java -Xmx200g -jar utils/ProgramFiles_Beagle5.4/beagle.22Jul22.46e.jar \
    gt=data/Control_GSE141901_Step3_${chr}.vcf.gz out=data/Control_GSE141901_Step3_imputed_${chr} \
    ref=utils/ProgramFiles_Beagle5.4/chr${chr}.1kg.phase3.v5a.b37.bref3 \
    map=utils/ProgramFiles_Beagle5.4/plink.chr${chr}.GRCh37.map \
    chrom="$chr" nthreads=20; done'

!bash -c 'for chr in {1..22}; do \
    java -Xmx200g -jar utils/ProgramFiles_Beagle5.4/beagle.22Jul22.46e.jar \
    gt=data/Control_GSE248483_Step3_${chr}.vcf.gz out=data/Control_GSE248483_Step3_imputed_${chr} \
    ref=utils/ProgramFiles_Beagle5.4/chr${chr}.1kg.phase3.v5a.b37.bref3 \
    map=utils/ProgramFiles_Beagle5.4/plink.chr${chr}.GRCh37.map \
    chrom="$chr" nthreads=20; done'

!bash -c 'for chr in {1..22}; do \
    java -Xmx200g -jar utils/ProgramFiles_Beagle5.4/beagle.22Jul22.46e.jar \
    gt=data/Control_GSE74100_Step3_${chr}.vcf.gz out=data/Control_GSE74100_Step3_imputed_${chr} \
    ref=utils/ProgramFiles_Beagle5.4/chr${chr}.1kg.phase3.v5a.b37.bref3 \
    map=utils/ProgramFiles_Beagle5.4/plink.chr${chr}.GRCh37.map \
    chrom="$chr" nthreads=20; done'

beagle.22Jul22.46e.jar (version 5.4)
Copyright (C) 2014-2022 Brian L. Browning
Enter "java -jar beagle.22Jul22.46e.jar" to list command line argument
Start time: 12:50 PM HKT on 15 Feb 2026

Command line: java -Xmx204800m -jar beagle.22Jul22.46e.jar
  gt=data/Case_GSE69664_GPL19864_Step3_1.vcf.gz
  out=data/Case_GSE69664_GPL19864_Step3_imputed_1
  ref=utils/ProgramFiles_Beagle5.4/chr1.1kg.phase3.v5a.b37.bref3
  map=utils/ProgramFiles_Beagle5.4/plink.chr1.GRCh37.map
  chrom=1
  nthreads=20

Reference samples:                2,504
Study     samples:                  493

Window 1 [1:10177-18871042]
Reference markers:              225,316
Study     markers:                3,749

Burnin  iteration 1:           4 seconds
Burnin  iteration 2:           1 second
Burnin  iteration 3:           1 second

Estimated ne:                  530986
Estimated err:                 7.2e-04

Phasing iteration 1:           0 seconds
Phasing iteration 2:           0 seconds
Phasing iteration 3:           0 

## Post-imputation QC

In [9]:
# filter low quality imputation (dr2 > 0.8)
!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools view -i "DR2>0.8" -Oz data/Case_GSE69664_GPL19864_Step3_imputed_${chr}.vcf.gz > data/Case_GSE69664_GPL19864_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools view -i "DR2>0.8" -Oz data/Case_GSE69664_GPL20166_Step3_imputed_${chr}.vcf.gz > data/Case_GSE69664_GPL20166_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools view -i "DR2>0.8" -Oz data/Control_GSE141901_Step3_imputed_${chr}.vcf.gz > data/Control_GSE141901_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools view -i "DR2>0.8" -Oz data/Control_GSE248483_Step3_imputed_${chr}.vcf.gz > data/Control_GSE248483_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools view -i "DR2>0.8" -Oz data/Control_GSE74100_Step3_imputed_${chr}.vcf.gz > data/Control_GSE74100_Step3_imputed_filtered_${chr}.vcf.gz; done'

[W::vcf_parse] Contig '1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '2' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '3' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '4' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '5' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '6' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '7' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '8' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '9' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '10' is not defined in the header. (Quick workaroun

## Concat chromosomes

In [10]:
# indexing
!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools index -t data/Case_GSE69664_GPL19864_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools index -t data/Case_GSE69664_GPL20166_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools index -t data/Control_GSE141901_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools index -t data/Control_GSE248483_Step3_imputed_filtered_${chr}.vcf.gz; done'

!conda run -n snp_array bash -c 'for chr in {1..22}; do \
    bcftools index -t data/Control_GSE74100_Step3_imputed_filtered_${chr}.vcf.gz; done'

In [13]:
# concat

!conda run -n snp_array bash -c 'bcftools concat data/Case_GSE69664_GPL19864_Step3_imputed_filtered_{1..22}.vcf.gz -Oz -o data/Case_GSE69664_GPL19864_Step3_imputed_filtered_combined.vcf.gz'

!conda run -n snp_array bash -c 'bcftools concat data/Case_GSE69664_GPL20166_Step3_imputed_filtered_{1..22}.vcf.gz -Oz -o data/Case_GSE69664_GPL20166_Step3_imputed_filtered_combined.vcf.gz'

!conda run -n snp_array bash -c 'bcftools concat data/Control_GSE141901_Step3_imputed_filtered_{1..22}.vcf.gz -Oz -o data/Control_GSE141901_Step3_imputed_filtered_combined.vcf.gz'

!conda run -n snp_array bash -c 'bcftools concat data/Control_GSE248483_Step3_imputed_filtered_{1..22}.vcf.gz -Oz -o data/Control_GSE248483_Step3_imputed_filtered_combined.vcf.gz'

!conda run -n snp_array bash -c 'bcftools concat data/Control_GSE74100_Step3_imputed_filtered_{1..22}.vcf.gz -Oz -o data/Control_GSE74100_Step3_imputed_filtered_combined.vcf.gz'

Checking the headers and starting positions of 22 files
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_1.vcf.gz	28.499791 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_2.vcf.gz	30.185069 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_3.vcf.gz	26.670534 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_4.vcf.gz	26.802886 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_5.vcf.gz	23.237583 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_6.vcf.gz	25.642070 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_7.vcf.gz	22.069332 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_8.vcf.gz	20.128638 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_9.vcf.gz	16.442119 seconds
Concatenating data/Case_GSE69664_GPL19864_Step3_imputed_filtered_10.vcf.gz	19.202007 seconds
Concatenating data/Case

## Regenerate binary ped file & add phenotype

In [36]:
!conda run -n snp_array bash -c 'plink --vcf data/Case_GSE69664_GPL19864_Step3_imputed_filtered_combined.vcf.gz \
    --nonfounders --allow-no-sex \
    --double-id --biallelic-only strict \
    --make-bed --out data/Case_GSE69664_GPL19864_Step4'

!conda run -n snp_array bash -c 'plink --vcf data/Case_GSE69664_GPL20166_Step3_imputed_filtered_combined.vcf.gz \
    --nonfounders --allow-no-sex \
    --double-id --biallelic-only strict \
    --make-bed --out data/Case_GSE69664_GPL20166_Step4'

!conda run -n snp_array bash -c 'plink --vcf data/Control_GSE141901_Step3_imputed_filtered_combined.vcf.gz \
    --nonfounders --allow-no-sex \
    --double-id --biallelic-only strict \
    --make-bed --out data/Control_GSE141901_Step4'

!conda run -n snp_array bash -c 'plink --vcf data/Control_GSE248483_Step3_imputed_filtered_combined.vcf.gz \
    --nonfounders --allow-no-sex \
    --double-id --biallelic-only strict \
    --make-bed --out data/Control_GSE248483_Step4'

!conda run -n snp_array bash -c 'plink --vcf data/Control_GSE74100_Step3_imputed_filtered_combined.vcf.gz \
    --nonfounders --allow-no-sex \
    --double-id --biallelic-only strict \
    --make-bed --out data/Control_GSE74100_Step4'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/Case_GSE69664_GPL19864_Step4.log.
Options in effect:
  --allow-no-sex
  --biallelic-only strict
  --double-id
  --make-bed
  --nonfounders
  --out data/Case_GSE69664_GPL19864_Step4
  --vcf data/Case_GSE69664_GPL19864_Step3_imputed_filtered_combined.vcf.gz

362751 MB RAM detected; reserving 181375 MB for main workspace.
--vcf: data/Case_GSE69664_GPL19864_Step4-temporary.bed +
data/Case_GSE69664_GPL19864_Step4-temporary.bim +
data/Case_GSE69664_GPL19864_Step4-temporary.fam written.
(141797 variants skipped.)
8001261 variants loaded from .bim file.
493 people (0 males, 0 females, 493 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/Case_GSE69664_GPL19864_Step4.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 493 founders and 0 nonfounders present.
Calculating a

In [37]:
%%bash
mv data/Case_GSE69664_GPL19864_Step4.fam data/Case_GSE69664_GPL19864_Step4_NoPheno.fam
mv data/Case_GSE69664_GPL20166_Step4.fam data/Case_GSE69664_GPL20166_Step4_NoPheno.fam
mv data/Control_GSE141901_Step4.fam data/Control_GSE141901_Step4_NoPheno.fam
mv data/Control_GSE248483_Step4.fam data/Control_GSE248483_Step4_NoPheno.fam
mv data/Control_GSE74100_Step4.fam data/Control_GSE74100_Step4_NoPheno.fam

awk '{print "MHS", $2, $3, $4, $5, "2"}' data/Case_GSE69664_GPL19864_Step4_NoPheno.fam > data/Case_GSE69664_GPL19864_Step5.fam
awk '{print "MHS", $2, $3, $4, $5, "2"}' data/Case_GSE69664_GPL20166_Step4_NoPheno.fam > data/Case_GSE69664_GPL20166_Step5.fam
awk '{print "MHS", $2, $3, $4, $5, "1"}' data/Control_GSE141901_Step4_NoPheno.fam > data/Control_GSE141901_Step5.fam
awk '{print "MHS", $2, $3, $4, $5, "1"}' data/Control_GSE248483_Step4_NoPheno.fam > data/Control_GSE248483_Step5.fam
awk '{print "MHS", $2, $3, $4, $5, "1"}' data/Control_GSE74100_Step4_NoPheno.fam > data/Control_GSE74100_Step5.fam

rm data/Case_GSE69664_GPL19864_Step4_NoPheno.fam
rm data/Case_GSE69664_GPL20166_Step4_NoPheno.fam
rm data/Control_GSE141901_Step4_NoPheno.fam
rm data/Control_GSE248483_Step4_NoPheno.fam
rm data/Control_GSE74100_Step4_NoPheno.fam

In [38]:
%%bash
# Update bed and bim files to Step5
for dataset in Case_GSE69664_GPL19864 Case_GSE69664_GPL20166 Control_GSE141901 Control_GSE248483 Control_GSE74100; do
    cp data/${dataset}_Step4.bed data/${dataset}_Step5.bed
    cp data/${dataset}_Step4.bim data/${dataset}_Step5.bim
done

## Merge datasets

GSE74100 (control) and GSE69664_20166 (case) will be used for downstream analysis (training ml model)<br>
They were excluded here to avoid data leakage<br>
Both data are from the same genotyping platform

In [39]:
### merge controls
!conda run -n snp_array bash -c 'plink --bfile data/Control_GSE248483_Step5 \
    --extract data/Control_GSE141901_Step5.bim \
    --exclude utils/na_excluder.txt \
    --nonfounders --allow-no-sex \
    --make-bed --out data/Control_GSE248483_Step5_PreMerge'

!conda run -n snp_array bash -c 'plink --bfile data/Control_GSE141901_Step5 \
    --extract data/Control_GSE248483_Step5.bim \
    --exclude utils/na_excluder.txt \
    --nonfounders --allow-no-sex \
    --make-bed --out data/Control_GSE141901_Step5_PreMerge'

!conda run -n snp_array bash -c 'plink --bfile data/Control_GSE248483_Step5_PreMerge \
    --bmerge data/Control_GSE141901_Step5_PreMerge \
    --nonfounders --allow-no-sex \
    --make-bed --out data/GWAS_Control_Step5'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/Control_GSE248483_Step5_PreMerge.log.
Options in effect:
  --allow-no-sex
  --bfile data/Control_GSE248483_Step5
  --exclude utils/na_excluder.txt
  --extract data/Control_GSE141901_Step5.bim
  --make-bed
  --nonfounders
  --out data/Control_GSE248483_Step5_PreMerge

362751 MB RAM detected; reserving 181375 MB for main workspace.
6341397 variants loaded from .bim file.
86 people (0 males, 0 females, 86 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/Control_GSE248483_Step5_PreMerge.nosex .
86 phenotype values loaded from .fam.
--extract: 5085284 variants remaining.
--exclude: 5083211 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 86 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is exactly 1.


In [40]:
### merge case and controls
!conda run -n snp_array bash -c 'plink --bfile data/Case_GSE69664_GPL19864_Step5 \
    --extract data/GWAS_Control_Step5.bim \
    --exclude utils/na_excluder.txt \
    --nonfounders --allow-no-sex \
    --make-bed --out data/GWAS_Case_Step5_PreMerge'

!conda run -n snp_array bash -c 'plink --bfile data/GWAS_Control_Step5 \
    --extract data/Case_GSE69664_GPL19864_Step5.bim \
    --exclude utils/na_excluder.txt \
    --nonfounders --allow-no-sex \
    --make-bed --out data/GWAS_Control_Step5_PreMerge'

!conda run -n snp_array bash -c 'plink --bfile data/GWAS_Case_Step5_PreMerge \
    --bmerge data/GWAS_Control_Step5_PreMerge \
    --nonfounders --allow-no-sex \
    --make-bed --out data/GWAS_All_Step6'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/GWAS_Case_Step5_PreMerge.log.
Options in effect:
  --allow-no-sex
  --bfile data/Case_GSE69664_GPL19864_Step5
  --exclude utils/na_excluder.txt
  --extract data/GWAS_Control_Step5.bim
  --make-bed
  --nonfounders
  --out data/GWAS_Case_Step5_PreMerge

362751 MB RAM detected; reserving 181375 MB for main workspace.
8001261 variants loaded from .bim file.
493 people (0 males, 0 females, 493 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/GWAS_Case_Step5_PreMerge.nosex .
493 phenotype values loaded from .fam.
--extract: 4828562 variants remaining.
--exclude: 4828562 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 493 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is exactly 1.
4828562 variants and

In [41]:
# remove snps with multiple positions (informed by gwas_all.log)
!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step6 \
    --nonfounders --allow-no-sex \
    --list-duplicate-vars ids-only suppress-first \
    --out data/GWAS_All_Step6_DupVar'

!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step6 \
    --nonfounders --allow-no-sex \
    --exclude data/GWAS_All_Step6_DupVar.dupvar \
    --make-bed --out data/GWAS_All_Step7'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/GWAS_All_Step6_DupVar.log.
Options in effect:
  --allow-no-sex
  --bfile data/GWAS_All_Step6
  --list-duplicate-vars ids-only suppress-first
  --nonfounders
  --out data/GWAS_All_Step6_DupVar

362751 MB RAM detected; reserving 181375 MB for main workspace.
4828562 variants loaded from .bim file.
700 people (0 males, 0 females, 700 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/GWAS_All_Step6_DupVar.nosex .
700 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 700 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is exactly 1.
4828562 variants and 700 people pass filters and QC.
Among remaining phenotypes, 493 are cases and 207 are controls.
--list-duplicate-vars report written to dat

## QC for linkage disequalibrium

In [42]:
# Remove region of high linkage disequalibrium
# Source: https://genome.sph.umich.edu/wiki/Regions_of_high_linkage_disequilibrium_(LD)

!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step7 \
    --nonfounders --allow-no-sex \
    --make-set utils/high_ld_region.txt --write-set \
    --out data/GWAS_All_Step7_Highld'

!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step7 \
    --nonfounders --allow-no-sex \
    --exclude data/GWAS_All_Step7_Highld.set \
    --make-bed --out data/GWAS_All_Step8'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/GWAS_All_Step7_Highld.log.
Options in effect:
  --allow-no-sex
  --bfile data/GWAS_All_Step7
  --make-set utils/high_ld_region.txt
  --nonfounders
  --out data/GWAS_All_Step7_Highld
  --write-set

362751 MB RAM detected; reserving 181375 MB for main workspace.
4828552 variants loaded from .bim file.
700 people (0 males, 0 females, 700 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/GWAS_All_Step7_Highld.nosex .
700 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 700 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is exactly 1.
--make-set: 24 sets defined.
4828552 variants and 700 people pass filters and QC.
Among remaining phenotypes, 493 are cases and 207 are controls.
--write-se

In [44]:
# LD pruning

# clean up imputed snps that will not be included in downstream machine learning pipeline
!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step8 \
    --extract data/Case_GSE69664_GPL20166_Step3.bim \
    --nonfounders --allow-no-sex \
    --make-bed --out data/GWAS_All_Step8_PrePrune'

# pruning
!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step8_PrePrune \
    --indep-pairwise 200kb 1 0.5 \
    --nonfounders --allow-no-sex \
    --out data/GWAS_All_Step8_PrePrune_LD'

!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Step8_PrePrune \
    --extract data/GWAS_All_Step8_PrePrune_LD.prune.in \
    --nonfounders --allow-no-sex \
    --make-bed --out data/GWAS_All_Processed'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/GWAS_All_Step8_PrePrune.log.
Options in effect:
  --allow-no-sex
  --bfile data/GWAS_All_Step8
  --extract data/Case_GSE69664_GPL20166_Step3.bim
  --make-bed
  --nonfounders
  --out data/GWAS_All_Step8_PrePrune

362751 MB RAM detected; reserving 181375 MB for main workspace.
4586113 variants loaded from .bim file.
700 people (0 males, 0 females, 700 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/GWAS_All_Step8_PrePrune.nosex .
700 phenotype values loaded from .fam.
--extract: 436702 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 700 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is exactly 1.
436702 variants and 700 people pass filters and QC.
Among remaining phenotypes, 493 are cases and 207

## Association test

In [45]:
# using the first 10 pc as covariate to control for population stratification

# pca
!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Processed \
    --pca 10 \
    --nonfounders --allow-no-sex \
    --out data/GWAS_All_Processed_PCA'

# association test, 1-10 pcs as covariate
!conda run -n snp_array bash -c 'plink --bfile data/GWAS_All_Processed \
    --nonfounders --allow-no-sex \
    --covar data/GWAS_All_Processed_PCA.eigenvec --covar-number 1-10 \
    --logistic --adjust \
    --out data/GWAS_LogisticRegression'

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/GWAS_All_Processed_PCA.log.
Options in effect:
  --allow-no-sex
  --bfile data/GWAS_All_Processed
  --nonfounders
  --out data/GWAS_All_Processed_PCA
  --pca 10

362751 MB RAM detected; reserving 181375 MB for main workspace.
161092 variants loaded from .bim file.
700 people (0 males, 0 females, 700 ambiguous) loaded from .fam.
Ambiguous sex IDs written to data/GWAS_All_Processed_PCA.nosex .
700 phenotype values loaded from .fam.
Using up to 47 threads (change this with --threads).
Before main variant filters, 700 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is exactly 1.
161092 variants and 700 people pass filters and QC.
Among remaining phenotypes, 493 are cases and 207 are controls.
Relationship matrix calculation complete.
--pca: Results saved to data/GWAS_Al